## FCS Lab 1 Submission Report
- Name of Student: Lim Zhiyu, Kendrix
- Student ID: 1008152

### Command Line Instructions

For exercise 1: !python3 ex1.py -i sherlock_short.txt -o <file_name>.txt -k <any integer from 0 - 255> -m <"e" for encrypt, "d" for decrypt>

For exercise 2: !python3 ex2.py -i sherlock_short.txt -o <file_name>.txt -k <any integer from 0 - 255> -m <"e" for encrypt, "d" for decrypt>

For exercise 3: !python3 ex2.py -i flag -o <file_name>.png

### Expected Outputs for each exercise

#### Part 1: Shift Cipher for printable input

In [ ]:
!python3 ex1.py -i sherlock_short.txt -o textEncrypted.txt -k 10 -m e
!cat textEncrypted.txt

##### Expected Output:
    WB\4#roBvymu4RyvwoC@4Gry4GkC4ECEkvvI4FoBI4vkDo4sx4Dro4wyBxsxqC@4CkFo4Ezyx4DryCo4xyD4sxpBoAEoxD4ymmkCsyxC4Grox4ro4GkC4Ez4kvv4xsqrD@4GkC4CokDon4kD4Dro4lBokupkCD4Dklvo\

In [ ]:
!python3 ex1.py -i textEncrypted.txt -o textDecrypted.txt -k 10 -m d
!cat textDecrypted.txt

##### Expected Output:
    Mr. Sherlock Holmes, who was usually very late in the mornings, save upon those not infrequent occasions when he was up all night, was seated at the breakfast table.

#### Part 2: Shift Cipher for Binary Input

In [ ]:
!python ex2.py -i sherlock_short.txt -o binaryEncrypt.txt -k <any integer from 0 - 255> -m e
!cat binaryEncrypt.txt

##### Expected Output:
    W|8*]ro|vymu*Ryvwo}6*?ry*?k}*}kvv?*?o|?*vk~o*sx*~ro*wy|xsxq}6*}k?o*zyx*~ry}o*xy~*sxp|o{ox~*ymmk}syx}*?rox*ro*?k}*z*kvv*xsqr~6*?k}*}ok~on*k~*~ro*l|okupk}~*~klvo8

In [ ]:
!python ex2.py -i binaryEncrypt.txt -o binaryDecrypt.txt -k <any integer from 0 - 255> -m d
!cat binaryEncrypt.txt

##### Expected Output:
    Mr. Sherlock Holmes, who was usually very late in the mornings, save upon those not infrequent occasions when he was up all night, was seated at the breakfast table.

#### Part 3: Shift Cipher for Binary Input (flag)

In [ ]:
!python ex2.py -i flag -o flag_decrypted.png
!open flag_decrypted.png

##### Expected Output:
    The Switzerland flag

### Exercise 1 Explanation

In [ ]:
import sys
import argparse
import string

def doStuff(filein, fileout, key, mode):
    # open file handles to both files
    fin = open(filein, mode="r", encoding="utf-8", newline="\n")  # read mode
    fin_b = open(filein, mode="rb")  # binary read mode
    fout = open(fileout, mode="w", encoding="utf-8", newline="\n")  # write mode
    fout_b = open(fileout, mode="wb")  # binary write mode

    # create an empty string to hold the output after encrypting / decrypting
    result = ""
    # string module printable gives us a string that contains characters we want to shift amongst
    ALPHABET = string.printable
    # get the length of the string so that when we encrypt/decrypt, we can mod it to keep it within the string.printable
    N = len(ALPHABET)

    with open(filein, mode="r", encoding="utf-8", newline="\n") as fin:
        text = fin.read()
        '''
        if encryption is selected...
        (1) Iterate through the input string
        (2) For each of the characters in the input string,
            (2.1) check if the character is in ALPHABET (i.e. the string.printable)
            (2.2) if it is, get the index of that character in ALPHABET
            (2.3) perform encryption algorithm
            (2.4) add new encrypted char into the result string
        
        else if decryption is selected...
        The same as encryption just that the decryption algorithm is used in 2.3

        (3) then, after encrypting/decrypting, write onto the fout file with result
        '''
        if mode == "e":
            for char in text:
                if char in ALPHABET:
                    getCharIdx = ALPHABET.index(char)
                    newIdx = (getCharIdx + key) % N
                    encryptedChar = ALPHABET[newIdx]
                    result += encryptedChar
        else:
            for char in text:
                if char in ALPHABET:
                    getCharIdx = ALPHABET.index(char)
                    newIdx = (getCharIdx - key) % N
                    encryptedChar = ALPHABET[newIdx]
                    result += encryptedChar

        fout.write(result)

### Exercise 2 Explanation

In [ ]:
import sys
import argparse
import string


def doStuff(filein, fileout, key, mode):
    # open file handles to both files
    fin = open(filein, mode="r", encoding="utf-8", newline="\n")  # read mode
    fin_b = open(filein, mode="rb")  # binary read mode
    fout = open(fileout, mode="w", encoding="utf-8", newline="\n")  # write mode
    fout_b = open(fileout, mode="wb")  # binary write mode

    PNG = [137, 80, 78, 71, 13, 10, 26, 10]

    with open(filein, mode="rb") as fin_b:
        binaryInput = fin_b.read()
        # create a bytearray with the binary input
        ba = bytearray(binaryInput)
        # take the first 8 bytes of the array to check for png file
        byteHeader = ba[:8]

        if mode != None:

            for char in ba:
                # each char is now an int
                if mode == "e":
                    shiftedChar = (char + key) % 256
                    fout_b.write(bytes([shiftedChar]))
                elif mode == "d":
                    shiftedChar = (char - key) % 256
                    fout_b.write(bytes([shiftedChar]))
        else:
            # iterate through every single possible key i.e. 0 - 255
            for possibleKey in range(256):
                # create an empty list to store every possible first 8 decrypted bytes
                decryptedBytes = []
                # use this key to decrypt the first 8 bytes
                for byte in byteHeader:
                    decryptedBytes.append((byte - possibleKey) % 256)
                    # then, we check if the current combination of the decrypted 8 bytes fits
                    # the PNG file special 8 byte combination
                if decryptedBytes == PNG:
                    print(f"The correct key has been found... {possibleKey}")
                    break

            # if indeed the correct key, we will use it to decrypt the whole file
            for byte in ba:
                shiftedChar = (byte - possibleKey) % 256
                fout_b.write(bytes([shiftedChar]))

Breakdown of the concept used for exercise 2 (binary):
1) We read the input file (i.e. sherlock_short.txt) as a binary input
2) We then create a bytearray with the binary input
3) Iterate through the bytearray
    3.1) For every byte in the array,
        3.1.1) if its encryption,
            3.1.a) add the key to the byte and make sure to modulus 256 to prevent the result value from getting out of the 0 - 255 range
            3.1.b) write the result into the output file, making sure to utilise the bytes() method to convert the result to a byte as we writing in binary back to the output file
        3.1.2) if its decryption,
            3.1.a) same thing as encryption just that the algorihm for decryption is to substract the key from each byte

Breakdown of the concepts used for exercise 2 (flag):
1) From the lab notes, we know that the output is a PNG file
- This allows us to know that every PNG file starts off with the same 8 bytes (137, 80, 78, 71, 13, 10, 26, 10)
2) Created a list with this 8 bytes labelled PNG
3) The whole idea of this code is to decrypt the binary input's first 8 bytes using every single possible key starting from 0 up to 255
- Once the right key is found, we proceed decrypting the rest of the file with that correct key

P.S. The mode to decrypt the flag file is None, no mode input is needed for the command to run